# IPC Toolkit Python Examples

In [1]:
import numpy
import scipy
import scipy.linalg
import plotly.figure_factory as ff
import plotly.graph_objects as go
import meshio

import sys
import pathlib
sys.path.append(str(pathlib.Path("").resolve().parent / "build" / "release" / "python"))  # noqa

from ipctk import *

## Distance

In [2]:
numpy.random.seed(0)
ea0, ea1, eb0, eb1 = (numpy.random.random(3) for i in range(4))
edge_edge_distance(ea0, ea1, eb0, eb1)

0.03313444539727968

In [3]:
e0 = numpy.array([-1.0, -1.0])
e1 = numpy.array([1.0, 1.0])
xs = numpy.linspace(-2, 2, 100)
ys = numpy.linspace(-2, 2, 100)
dtypes = numpy.empty((xs.size, ys.size), dtype=int)
for i, x in enumerate(xs):
    for j, y in enumerate(ys):
        dtypes[j, i] = point_edge_distance_type(
            numpy.array([x, y]), e0, e1)
fig = go.Figure(data=[
    go.Scatter(x=[e0[0], e1[0]], y=[e0[1], e1[1]]),
    go.Contour(z=dtypes, x=xs, y=ys)
], layout=go.Layout(width=400, height=400))
fig.show()

In [4]:
e0 = numpy.array([-1, -1], dtype=float)
e1 = numpy.array([1, 1], dtype=float)
xs = numpy.linspace(-2, 2, 100)
ys = numpy.linspace(-2, 2, 100)
distances = numpy.empty((xs.size, ys.size))
for i, x in enumerate(xs):
    for j, y in enumerate(ys):
        distances[j, i] = point_edge_distance(numpy.array([x, y]), e0, e1)
fig = go.Figure(data=[
    go.Scatter(x=[e0[0], e1[0]], y=[e0[1], e1[1]]),
    go.Contour(z=numpy.sqrt(distances), x=xs, y=ys)
], layout=go.Layout(width=800, height=800))
fig.show()

In [36]:
e0 = numpy.array([-1, -1], dtype=float)
e1 = numpy.array([1, 1], dtype=float)
x, y = numpy.meshgrid(numpy.arange(-2, 2, .2), numpy.arange(-2, 2, .2))
u, v = numpy.empty(x.shape), numpy.empty(x.shape)
d = numpy.empty(x.shape)
for i in range(x.shape[0]):
    for j in range(x.shape[1]):
        grad = point_edge_distance_gradient(
            numpy.array([x[i, j], y[i, j]]), e0, e1)
        u[i, j] = grad[0]
        v[i, j] = grad[1]
        d[i, j] = numpy.sqrt(point_edge_distance(
            numpy.array([x[i, j], y[i, j]]), e0, e1))

fig = ff.create_quiver(x, y, u, v, name="point_grad")
fig.add_trace(go.Scatter(x=[e0[0], e1[0]], y=[e0[1], e1[1]], name="edge"))
fig.update_layout(width=800, height=800)
fig.show()
fig = ff.create_quiver(x, y, u/(2 * d), v / (2 * d), name="point_grad")
fig.add_trace(go.Scatter(x=[e0[0], e1[0]], y=[e0[1], e1[1]], name="edge"))
fig.update_layout(width=800, height=800)
fig.show()

/var/folders/y1/l7pg454j0jv6b028zqg1frsc0000gn/T/ipykernel_56374/2089789646.py:19: RuntimeWarning:

invalid value encountered in divide



In [33]:
e0 = numpy.array([-1, -1], dtype=float)
e1 = numpy.array([1, 1], dtype=float)
x, y = numpy.meshgrid(numpy.arange(-2, 2, .2), numpy.arange(-2, 2, .2))
d_xx, d_xy, d_yx, d_yy = [numpy.empty(x.shape) for i in range(4)]
for i in range(x.shape[0]):
    for j in range(x.shape[1]):
        hess = point_edge_distance_hessian(
            numpy.array([x[i, j], y[i, j]]), e0, e1)
        d_xx[i, j] = hess[0, 0]
        d_xy[i, j] = hess[0, 1]
        d_yx[i, j] = hess[1, 0]
        d_yy[i, j] = hess[1, 1]

fig = ff.create_quiver(x, y, d_xx, d_xy, name="point_hessian_x")
fig.add_traces(ff.create_quiver(x, y, d_yx, d_yy, name="point_hessian_x").data)
fig.add_trace(go.Scatter(x=[e0[0], e1[0]], y=[e0[1], e1[1]], name="edge"))
fig.update_layout(width=800, height=800)
fig.show()

In [6]:
p = numpy.array([0, 0], dtype=float)
xs = numpy.linspace(-2, 2, 100)
ys = numpy.linspace(-2, 2, 100)
distances = numpy.empty((xs.size, ys.size))
for i, x in enumerate(xs):
    for j, y in enumerate(ys):
        distances[j, i] = point_point_distance(numpy.array([x, y]), p)
fig = go.Figure(data=[
    go.Scatter(x=[p[0]], y=[p[1]]),
    go.Contour(z=numpy.sqrt(distances), x=xs, y=ys)
], layout=go.Layout(width=800, height=800))
fig.show()

In [7]:
p = numpy.array([0, 0], dtype=float)
x, y = numpy.meshgrid(numpy.arange(-2, 2, .2), numpy.arange(-2, 2, .2))
u, v = numpy.empty(x.shape), numpy.empty(x.shape)
d = numpy.empty(x.shape)
for i in range(x.shape[0]):
    for j in range(x.shape[1]):
        grad = point_point_distance_gradient(
            numpy.array([x[i, j], y[i, j]]), p)
        u[i, j] = grad[0]
        v[i, j] = grad[1]
        d[j, i] = numpy.sqrt(point_point_distance(
            numpy.array([x[i, j], y[i, j]]), p))

fig = ff.create_quiver(x, y, u, v, name="point_grad")
fig.add_trace(go.Scatter(x=[p[0]], y=[p[1]], name="edge"))
fig.update_layout(width=800, height=800)
fig.show()
fig = ff.create_quiver(x, y, u/(2 * d), v / (2 * d), name="point_grad")
fig.add_trace(go.Scatter(x=[p[0]], y=[p[1]], name="edge"))
fig.update_layout(width=800, height=800)
fig.show()

In [8]:
ea0 = numpy.random.random(3)
ea1 = numpy.random.random(3)
eb0 = numpy.random.random(3)
eb1 = numpy.random.random(3)

print(edge_edge_distance(ea0, ea1, eb0, eb1))
print(edge_edge_distance_gradient(ea0, ea1, eb0, eb1))
print(edge_edge_distance_hessian(ea0, ea1, eb0, eb1))

0.3298505342009098
[-0.46155054 -0.00707979 -0.2998706  -0.50158084 -0.00769382 -0.3258784
  0.          0.          0.          0.96313139  0.01477362  0.625749  ]
[[ 0.3819837   0.13132058 -0.20871499  0.72453323  0.14745627 -0.02578582
   0.          0.          0.         -1.10651693 -0.27877685  0.23450081]
 [ 0.13132058  0.23625808  0.35449442 -0.38282811  0.24868745  0.04379623
   0.          0.          0.          0.25150752 -0.48494553 -0.39829065]
 [-0.20871499  0.35449442 -0.10411631  0.60844965  0.39805202  0.42952853
   0.          0.          0.         -0.39973466 -0.75254644 -0.32541222]
 [ 0.72453323 -0.38282811  0.60844965 -0.11465484 -0.42986714  0.07517129
   0.          0.          0.         -0.60987839  0.81269524 -0.68362094]
 [ 0.14745627  0.24868745  0.39805202 -0.42986714  0.2612043   0.04917758
   0.          0.          0.          0.28241087 -0.50989175 -0.4472296 ]
 [-0.02578582  0.04379623  0.42952853  0.07517129  0.04917758  0.53382666
   0.          0

In [9]:
e0 = numpy.array([-1, -1], dtype=float)
e1 = numpy.array([1, 1], dtype=float)
xs = numpy.linspace(-2, 2, 100)
ys = numpy.linspace(-2, 2, 100)
distances = numpy.empty((xs.size, ys.size))
for i, x in enumerate(xs):
    for j, y in enumerate(ys):
        distances[j, i] = point_line_distance(numpy.array([x, y]), e0, e1)
fig = go.Figure(data=[
    go.Scatter(x=[e0[0], e1[0]], y=[e0[1], e1[1]]),
    go.Contour(z=numpy.sqrt(distances), x=xs, y=ys)
], layout=go.Layout(width=800, height=800))
fig.show()

### Edge-Edge Distance Mollification

In [10]:
x = numpy.linspace(0, 1e-9)
y = numpy.asarray(list(map(lambda x: edge_edge_mollifier(x, eps_x=1), x)))
fig = go.Figure(data=go.Scatter(x=x, y=y))
fig.show()

## IPC Barrier Function

In [11]:
d = numpy.geomspace(1e-12, 0.2, 1000)
dhat = 0.1
b = numpy.vectorize(lambda x: barrier(x, dhat))(d)
b_grad = numpy.vectorize(lambda x: barrier_gradient(x, dhat))(d)
b_hess = numpy.vectorize(lambda x: barrier_hessian(x, dhat))(d)
fig = go.Figure(data=[go.Scatter(x=d, y=b, name="b(x)"), go.Scatter(x=d, y=b_grad, name=r"\nabla b(x)"), go.Scatter(x=d, y=b_hess, name=r"\nabla^2 b(x)")])
fig.update_layout(yaxis_range=[-1000, 1000])
fig.show()

## Compute Barrier Potential

In [12]:
V = numpy.random.random((4, 3))
E = numpy.arange(10).reshape(-1, 2)
F = numpy.arange(9).reshape(-1, 3)
dhat = 1

In [13]:
vvc = VertexVertexConstraint(0, 1)
vvc.weight = 10
print("potential:", vvc.compute_potential(V, E, F, dhat))
print("gradient:", vvc.compute_potential_gradient(V, E, F, dhat))
print("hessian:", vvc.compute_potential_hessian(V, E, F, dhat, False))
print("weight:", vvc.weight)

potential: 0.13736206524830097
gradient: [ 3.10836254 -0.44411327  1.02048799 -3.10836254  0.44411327 -1.02048799]
hessian: [[ 45.4591324   -7.03247155  16.15928481 -45.4591324    7.03247155
  -16.15928481]
 [ -7.03247155  -2.75657651  -2.30878884   7.03247155   2.75657651
    2.30878884]
 [ 16.15928481  -2.30878884   1.54380415 -16.15928481   2.30878884
   -1.54380415]
 [-45.4591324    7.03247155 -16.15928481  45.4591324   -7.03247155
   16.15928481]
 [  7.03247155   2.75657651   2.30878884  -7.03247155  -2.75657651
   -2.30878884]
 [-16.15928481   2.30878884  -1.54380415  16.15928481  -2.30878884
    1.54380415]]
weight: 10.0


In [14]:
cs = Candidates()
num_candidates = 20
cs.ev_candidates = [EdgeVertexCandidate(i, i+1) for i in range(num_candidates)]
assert(len(cs) == num_candidates)
cs.ev_candidates

[EdgeVertexCandidate(0, 1),
 EdgeVertexCandidate(1, 2),
 EdgeVertexCandidate(2, 3),
 EdgeVertexCandidate(3, 4),
 EdgeVertexCandidate(4, 5),
 EdgeVertexCandidate(5, 6),
 EdgeVertexCandidate(6, 7),
 EdgeVertexCandidate(7, 8),
 EdgeVertexCandidate(8, 9),
 EdgeVertexCandidate(9, 10),
 EdgeVertexCandidate(10, 11),
 EdgeVertexCandidate(11, 12),
 EdgeVertexCandidate(12, 13),
 EdgeVertexCandidate(13, 14),
 EdgeVertexCandidate(14, 15),
 EdgeVertexCandidate(15, 16),
 EdgeVertexCandidate(16, 17),
 EdgeVertexCandidate(17, 18),
 EdgeVertexCandidate(18, 19),
 EdgeVertexCandidate(19, 20)]

In [15]:
assert(EdgeVertexCandidate(0, 1) == EdgeVertexCandidate(0, 1))

In [16]:
help(CollisionConstraints.build)

Help on instancemethod in module ipctk:

build(...)
    build(*args, **kwargs)
    Overloaded function.
    
    1. build(self: ipctk.CollisionConstraints, mesh: ipc::CollisionMesh, vertices: numpy.ndarray[numpy.float64[m, n]], dhat: float, dmin: float = 0, broad_phase_method: ipctk.BroadPhaseMethod = <BroadPhaseMethod.HASH_GRID: 1>) -> None
    
    
                Initialize the set of constraints used to compute the barrier potential.
    
                Parameters:
                    mesh: The collision mesh.
                    vertices: Vertices of the collision mesh.
                    dhat: The activation distance of the barrier.
                    dmin: Minimum distance.
                    broad_phase_method: Broad-phase method to use.
                
    
    2. build(self: ipctk.CollisionConstraints, candidates: ipctk.Candidates, mesh: ipc::CollisionMesh, vertices: numpy.ndarray[numpy.float64[m, n]], dhat: float, dmin: float = 0) -> None
    
    
                Init

## Real-World Example

In [17]:
in_mesh = meshio.read("../tests/data/bunny.obj")
V = in_mesh.points
F = in_mesh.cells[0].data
E = edges(F)

In [18]:
mesh = CollisionMesh(V, E, F)

In [19]:
U = numpy.random.random((mesh.num_vertices, 3))
V1 = mesh.displace_vertices(0.01 * U)
has_intersections(mesh, V1)

True

In [20]:
import meshplot
meshplot.plot(mesh.rest_positions, mesh.faces)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0021269…

In [21]:
cs = CollisionConstraints()
cs.build(mesh, V, 1e-2)

In [22]:
len(cs)

606

In [23]:
cs.compute_potential(mesh, V, 1e-2)

3.0948257658451036e-06

In [24]:
g = cs.compute_potential_gradient(mesh, V, 1e-2)
numpy.linalg.norm(g)

0.0003114269379742743

In [25]:
H = cs.compute_potential_hessian(mesh, V, 1e-2)
numpy.linalg.norm(H.todense())

0.2680460874504235

## Intersection Check

In [26]:
has_intersections(mesh, V)

False

In [27]:
is_step_collision_free(mesh, V, 2*V)

True

In [28]:
is_step_collision_free(mesh, V, -V)

False

In [29]:
a = compute_collision_free_stepsize(mesh, V, -V)
a

0.4489593505859375

In [30]:
is_step_collision_free(mesh, V, (1.0 - a)*V - a*V)

True